In [ ]:
!unzip -q /content/drive/MyDrive/tabular-playground-series-jun-2021.zip

In [ ]:
!pip install -U lightautoml


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.5/262.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 34.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of lightautoml to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.9 MB/s eta 0:00:00
     

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.dataset.roles import NumericRole

In [ ]:
N_THREADS = 4 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 8 * 3600 # Time in seconds for automl run
TARGET_NAME = 'target'

In [ ]:
%%time

train_data = pd.read_csv('/content/LightAutoML/train.csv')
train_data[TARGET_NAME] = train_data[TARGET_NAME].str.slice(start=6).astype(int) - 1
train_data.head()

In [ ]:
%%time

train_data = pd.read_csv('/content/LightAutoML/train.csv')
train_data[TARGET_NAME] = train_data[TARGET_NAME].str.slice(start=6).astype(int) - 1
train_data.head()

CPU times: user 916 ms, sys: 171 ms, total: 1.09 s
Wall time: 1.11 s


,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,...,0,0,0,0,0,0,2,0,0,5
1,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,5
2,2,0,0,0,0,0,1,0,3,0,...,0,0,0,0,1,0,0,0,0,1
3,3,0,0,7,0,1,5,2,2,0,...,0,4,0,2,2,0,4,3,0,7
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
test_data = pd.read_csv('/content/LightAutoML/test.csv')
test_data.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,200000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,200001,1,2,0,0,0,0,0,0,0,...,3,1,3,0,0,0,0,3,0,0
2,200002,0,1,7,1,0,0,0,0,6,...,3,0,0,0,0,3,0,2,0,0
3,200003,0,0,0,4,3,1,0,0,0,...,0,0,0,1,0,0,0,4,0,0
4,200004,0,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
submission = pd.read_csv('/content/LightAutoML/sample_submission.csv')
submission.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
1,200001,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
2,200002,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
3,200003,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
4,200004,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111


In [ ]:
%%time

task = Task('multiclass',)

CPU times: user 5.27 ms, sys: 0 ns, total: 5.27 ms
Wall time: 11.5 ms


In [ ]:
%%time

roles = {
    'target': TARGET_NAME,
    'drop': ['id'],
}

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.34 µs


In [ ]:
%%time

automl = TabularAutoML(task = task,
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {
                           'use_algos': [['linear_l2', 'lgb_tuned', 'cb_tuned']],
                           'return_all_predictions': True,
                           'weighted_blender_max_nonzero_coef': 0.0
                       },
                       tuning_params = {'max_tuning_time': 1800},
                       reader_params = {'n_jobs': N_THREADS},
                       config_path = '/content/conf_0_sel_type_0.yml'
                       )
oof_pred = automl.fit_predict(train_data, roles = roles)
print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
DEBUG:lightautoml.ml_algo.boost_cb:1323:	learn: 1.7221667	test: 1.7442286	best: 1.7442244 (1319)	total: 8m 14s	remaining: 10m 25s
DEBUG:lightautoml.ml_algo.boost_cb:1324:	learn: 1.7221528	test: 1.7442283	best: 1.7442244 (1319)	total: 8m 14s	remaining: 10m 24s
DEBUG:lightautoml.ml_algo.boost_cb:1325:	learn: 1.7221368	test: 1.7442270	best: 1.7442244 (1319)	total: 8m 14s	remaining: 10m 24s
DEBUG:lightautoml.ml_algo.boost_cb:1326:	learn: 1.7221147	test: 1.7442241	best: 1.7442241 (1326)	total: 8m 14s	remaining: 10m 23s
DEBUG:lightautoml.ml_algo.boost_cb:1327:	learn: 1.7220994	test: 1.7442240	best: 1.7442240 (1327)	total: 8m 15s	remaining: 10m 23s
DEBUG:lightautoml.ml_algo.boost_cb:1328:	learn: 1.7220809	test: 1.7442200	best: 1.7442200 (1328)	total: 8m 15s	remaining: 10m 22s
DEBUG:lightautoml.ml_algo.boost_cb:1329:	learn: 1.7220616	test: 1.7442152	best: 1.7442152 (1329)	total: 8m 15s	remaining: 10m 22s
DEBUG:lightautoml.ml_algo.boost_cb:1330:	learn: 1.7220

oof_pred:
array([[0.02796169, 0.03028661, 0.03006789, 0.01438011, 0.00947335,
        0.5576299 , 0.04937352, 0.20422892, 0.07659801, 0.01868347,
        0.01879015, 0.01720787, 0.01278002, 0.00428057, 0.7326314 ,
        0.03678478, 0.1197086 , 0.03913317, 0.01800114, 0.01847242,
        0.01626847, 0.00792821, 0.00395746, 0.7527922 , 0.03028822,
        0.11233543, 0.03995647],
       [0.03995177, 0.17910598, 0.08901282, 0.02139092, 0.01201478,
        0.31262743, 0.05082793, 0.1789063 , 0.1161621 , 0.02751461,
        0.1180456 , 0.07131096, 0.02280855, 0.00724527, 0.5135903 ,
        0.02888307, 0.10624492, 0.10435674, 0.04078497, 0.14072625,
        0.07780802, 0.02381426, 0.00982466, 0.44662842, 0.03645484,
        0.11533985, 0.10861874],
       [0.04805271, 0.13609987, 0.08651937, 0.02457317, 0.01483565,
        0.22851342, 0.07188337, 0.24159501, 0.14792746, 0.05766048,
        0.1065139 , 0.07213455, 0.02632761, 0.01536652, 0.16237143,
        0.09640066, 0.2857569 , 0.177467

In [ ]:
columns = ['PREDS_Linear_' + str(i) for i in range(1, 10)] + \
          ['PREDS_LGBM_' + str(i) for i in range(1, 10)] + \
          ['PREDS_CB_' + str(i) for i in range(1, 10)] + \
          ['PREDS_NN_' + str(i) for i in range(1, 10)]
columns

['PREDS_Linear_1',
 'PREDS_Linear_2',
 'PREDS_Linear_3',
 'PREDS_Linear_4',
 'PREDS_Linear_5',
 'PREDS_Linear_6',
 'PREDS_Linear_7',
 'PREDS_Linear_8',
 'PREDS_Linear_9',
 'PREDS_LGBM_1',
 'PREDS_LGBM_2',
 'PREDS_LGBM_3',
 'PREDS_LGBM_4',
 'PREDS_LGBM_5',
 'PREDS_LGBM_6',
 'PREDS_LGBM_7',
 'PREDS_LGBM_8',
 'PREDS_LGBM_9',
 'PREDS_CB_1',
 'PREDS_CB_2',
 'PREDS_CB_3',
 'PREDS_CB_4',
 'PREDS_CB_5',
 'PREDS_CB_6',
 'PREDS_CB_7',
 'PREDS_CB_8',
 'PREDS_CB_9',
 'PREDS_NN_1',
 'PREDS_NN_2',
 'PREDS_NN_3',
 'PREDS_NN_4',
 'PREDS_NN_5',
 'PREDS_NN_6',
 'PREDS_NN_7',
 'PREDS_NN_8',
 'PREDS_NN_9']

In [ ]:
test_pred = automl.predict(test_data)
print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

Prediction for test data:
array([[0.04032942, 0.21022995, 0.10034822, 0.02155248, 0.01201997,
        0.2714866 , 0.04977208, 0.17371431, 0.120547  , 0.0539349 ,
        0.38567728, 0.17059542, 0.02306439, 0.01447292, 0.16133553,
        0.02468969, 0.05257523, 0.11365465, 0.05412539, 0.35379165,
        0.14973931, 0.02700206, 0.01251281, 0.17920385, 0.02803543,
        0.06854689, 0.12704262],
       [0.04830809, 0.16471593, 0.09454039, 0.02460317, 0.01507456,
        0.21585491, 0.06813296, 0.22034028, 0.1484297 , 0.03897237,
        0.09887565, 0.07246157, 0.01743269, 0.01471703, 0.25533962,
        0.07313459, 0.31546608, 0.11360043, 0.04274221, 0.07384145,
        0.05629124, 0.01844389, 0.01288822, 0.26286536, 0.08581824,
        0.31256726, 0.1345421 ],
       [0.04080669, 0.0940927 , 0.06194054, 0.02037482, 0.01275984,
        0.38573828, 0.05836351, 0.21266499, 0.11325872, 0.02174406,
        0.03263529, 0.02212304, 0.01144029, 0.00574426, 0.7033652 ,
        0.02931079, 0.11

In [ ]:
oof_nn = pd.read_csv('/content/OOF_nn_predict.csv')
test_nn = pd.read_csv('/content/nn_predict.csv')

In [ ]:
OOF_preds = pd.DataFrame(np.hstack((oof_pred.data, oof_nn.drop('id', axis = 1))), columns = columns)
TEST_preds = pd.DataFrame(np.hstack((test_pred.data, test_nn.drop('id', axis = 1))), columns = columns)

In [ ]:
for col in columns:
    train_data[col] = OOF_preds[col]
    test_data[col] = TEST_preds[col]

In [ ]:
%%time

roles = {
    'target': TARGET_NAME,
    'drop': ['id'],
    NumericRole(np.float32, prob = True): columns
}

automl = TabularUtilizedAutoML(task = task,
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {
                           'use_algos': [['lgb_tuned', 'cb_tuned']],
                       },
                       tuning_params = {'max_tuning_time': 1800},
                       reader_params = {'n_jobs': N_THREADS},
                       configs_list = ['/content/conf_1_sel_type_1.yml'],
                       max_runs_per_config=1
                       )
oof_pred = automl.fit_predict(train_data, roles = roles)
print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
DEBUG:lightautoml.ml_algo.boost_cb:126:	learn: 1.7383052	test: 1.7490100	best: 1.7490100 (126)	total: 1m 23s	remaining: 42m 14s
DEBUG:lightautoml.ml_algo.boost_cb:127:	learn: 1.7381958	test: 1.7489638	best: 1.7489638 (127)	total: 1m 23s	remaining: 42m 6s
DEBUG:lightautoml.ml_algo.boost_cb:128:	learn: 1.7380405	test: 1.7489238	best: 1.7489238 (128)	total: 1m 24s	remaining: 42m 2s
DEBUG:lightautoml.ml_algo.boost_cb:129:	learn: 1.7379398	test: 1.7488683	best: 1.7488683 (129)	total: 1m 24s	remaining: 41m 56s
DEBUG:lightautoml.ml_algo.boost_cb:130:	learn: 1.7377958	test: 1.7488443	best: 1.7488443 (130)	total: 1m 25s	remaining: 41m 51s
DEBUG:lightautoml.ml_algo.boost_cb:131:	learn: 1.7376684	test: 1.7488236	best: 1.7488236 (131)	total: 1m 25s	remaining: 41m 44s
DEBUG:lightautoml.ml_algo.boost_cb:132:	learn: 1.7375680	test: 1.7487736	best: 1.7487736 (132)	total: 1m 25s	remaining: 41m 37s
DEBUG:lightautoml.ml_algo.boost_cb:133:	learn: 1.7374646	test: 1.74870

oof_pred:
array([[0.0171603 , 0.01700712, 0.01089593, 0.00811554, 0.00495662,
        0.7861609 , 0.02117713, 0.10406105, 0.03046545],
       [0.026956  , 0.09476013, 0.05313437, 0.01450186, 0.00794464,
        0.58606637, 0.02945275, 0.10988447, 0.07729942],
       [0.04953931, 0.10003357, 0.07887594, 0.03028681, 0.02255646,
        0.17275245, 0.09596284, 0.28821376, 0.16177884],
       [0.05919861, 0.12937821, 0.08872187, 0.03476667, 0.02848674,
        0.10552169, 0.09440112, 0.28059277, 0.17893231],
       [0.05692322, 0.32497793, 0.14526957, 0.0299771 , 0.01586055,
        0.13967706, 0.03931858, 0.09331092, 0.15468507],
       [0.04894884, 0.16238467, 0.10670397, 0.02816184, 0.01958862,
        0.05071045, 0.09219819, 0.31044254, 0.1808609 ],
       [0.04177968, 0.08007368, 0.0621475 , 0.02426387, 0.01562832,
        0.2846148 , 0.08293214, 0.2694955 , 0.13906452],
       [0.05329153, 0.04212771, 0.03721558, 0.02578265, 0.01803607,
        0.28338194, 0.0991272 , 0.32197273, 0.1

In [ ]:
test_pred = automl.predict(test_data)
print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

print('Check scores...')
print('OOF score: {}'.format(log_loss(train_data[TARGET_NAME].values, oof_pred.data)))

Prediction for test data:
array([[0.0586983 , 0.415325  , 0.15385914, 0.02613389, 0.0120671 ,
        0.14780807, 0.02164085, 0.04739936, 0.11706831],
       [0.04293721, 0.06839815, 0.05370767, 0.01941224, 0.0146467 ,
        0.27927345, 0.08641627, 0.30686048, 0.1283478 ],
       [0.01842751, 0.0253447 , 0.01532078, 0.00927434, 0.00550239,
        0.74706525, 0.02505793, 0.11271154, 0.04129555],
       [0.04844346, 0.11584409, 0.08386932, 0.0344711 , 0.01739933,
        0.23495072, 0.08349097, 0.2144724 , 0.16705862],
       [0.04179064, 0.11261109, 0.0845699 , 0.02572553, 0.01614843,
        0.2852397 , 0.06612457, 0.21745048, 0.15033968],
       [0.04645716, 0.1966175 , 0.10294554, 0.02627151, 0.01344527,
        0.28889567, 0.04834565, 0.13580129, 0.14122045],
       [0.04283496, 0.09986952, 0.07920669, 0.02983782, 0.01843034,
        0.22974047, 0.08869132, 0.25358924, 0.15779965],
       [0.0402532 , 0.44090974, 0.19733277, 0.02170994, 0.01433108,
        0.03160896, 0.03615557,

In [ ]:
submission.iloc[:, 1:] = test_pred.data
submission.to_csv('/content/lightautoml_2lvl_8hours_with_nn_oofs.csv', index = False)